In [ ]:
# MBTA System Challenge - Income Analysis

This notebook analyzes income distribution in areas within 1 mile of MBTA train stations.

**Counties:** Suffolk, Middlesex, and Norfolk  
**Focus:** Block groups within 1-mile buffer of train stations


## 1. Load Libraries


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as ctx
from pathlib import Path
import numpy as np

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# IMPORTANT: Use inline backend to show plots in Jupyter
%matplotlib inline
plt.ion()  # Turn on interactive mode


## 2. Load Merged Income Data


In [ ]:
# Load the merged income data (from step1_merge_acs_income.py)
data_file = 'data/neighborhood_income_merged.geojson'

if Path(data_file).exists():
    gdf = gpd.read_file(data_file)
    print(f"✅ Loaded {len(gdf)} block groups")
    print(f"Columns: {gdf.columns.tolist()}")
    print(f"\nIncome Statistics:")
    print(f"  Mean: ${gdf['median_household_income'].mean():,.0f}")
    print(f"  Median: ${gdf['median_household_income'].median():,.0f}")
    print(f"  Min: ${gdf['median_household_income'].min():,.0f}")
    print(f"  Max: ${gdf['median_household_income'].max():,.0f}")
else:
    print(f"❌ File not found: {data_file}")
    print("Please run step1_merge_acs_income.py first")


## 3. Display Sample Data


In [ ]:
# Show first 10 rows
if 'gdf' in locals():
    sample_data = gdf[['GEOID', 'median_household_income']].head(10)
    print("Sample Data (First 10 Block Groups):")
    print("=" * 60)
    display(sample_data)
    
    # Show data without geometry
    print("\n\nFull Data Summary (without geometry):")
    print("=" * 60)
    display(gdf.drop(columns='geometry').head(20))


## 4. Load Train Stations


In [ ]:
# Load train stations from GTFS data
from shapely.geometry import Point

gtfs_stops_file = 'data/gtfs/stops.txt'
gtfs_routes_file = 'data/gtfs/routes.txt'

train_stations_gdf = None

if Path(gtfs_stops_file).exists():
    stops_df = pd.read_csv(gtfs_stops_file)
    stops_df = stops_df[stops_df['stop_lat'].notna() & stops_df['stop_lon'].notna()]
    
    # Filter for train stations (route_type: 0=Light rail, 1=Subway, 2=Rail)
    if Path(gtfs_routes_file).exists():
        routes_df = pd.read_csv(gtfs_routes_file)
        train_route_types = [0, 1, 2]
        train_routes = routes_df[routes_df['route_type'].isin(train_route_types)]['route_id'].unique()
        
        # Filter stops by name patterns (simplified approach)
        train_keywords = ['station', 'subway', 'red line', 'orange line', 'blue line', 
                         'green line', 'commuter rail', 'mbta', 't', 'line']
        if 'stop_name' in stops_df.columns:
            stops_df['is_train'] = stops_df['stop_name'].str.lower().str.contains(
                '|'.join(train_keywords), na=False
            )
            stops_df = stops_df[stops_df['is_train']]
    
    # Create Point geometries
    geometry = [Point(lon, lat) for lon, lat in zip(stops_df['stop_lon'], stops_df['stop_lat'])]
    train_stations_gdf = gpd.GeoDataFrame(stops_df, geometry=geometry, crs='EPSG:4326')
    
    print(f"✅ Loaded {len(train_stations_gdf)} train stations")
else:
    print(f"⚠️  GTFS stops file not found: {gtfs_stops_file}")


## 5. Bar Chart - Median Income by Block Groups


In [2]:
# Create bar chart showing median income by block groups
if 'gdf' in locals():
    # Sort by income and take top/bottom N for visualization
    gdf_sorted = gdf.sort_values('median_household_income', ascending=False)
    
    # Option 1: Top 20 and Bottom 20 block groups
    top_20 = gdf_sorted.head(20)
    bottom_20 = gdf_sorted.tail(20)
    
    fig, axes = plt.subplots(2, 1, figsize=(16, 12))
    
    # Top 20 block groups
    top_20['GEOID_short'] = top_20['GEOID'].str[-4:]  # Last 4 digits for readability
    axes[0].barh(range(len(top_20)), top_20['median_household_income'], 
                 color='#4ECDC4', edgecolor='black', linewidth=0.5)
    axes[0].set_yticks(range(len(top_20)))
    axes[0].set_yticklabels([f"BG {gid}" for gid in top_20['GEOID_short']], fontsize=8)
    axes[0].set_xlabel('Median Household Income ($)', fontsize=12)
    axes[0].set_title('Top 20 Block Groups by Median Income', fontsize=14, fontweight='bold')
    axes[0].grid(axis='x', alpha=0.3)
    axes[0].invert_yaxis()
    
    # Bottom 20 block groups
    bottom_20['GEOID_short'] = bottom_20['GEOID'].str[-4:]
    axes[1].barh(range(len(bottom_20)), bottom_20['median_household_income'], 
                 color='#FF6B6B', edgecolor='black', linewidth=0.5)
    axes[1].set_yticks(range(len(bottom_20)))
    axes[1].set_yticklabels([f"BG {gid}" for gid in bottom_20['GEOID_short']], fontsize=8)
    axes[1].set_xlabel('Median Household Income ($)', fontsize=12)
    axes[1].set_title('Bottom 20 Block Groups by Median Income', fontsize=14, fontweight='bold')
    axes[1].grid(axis='x', alpha=0.3)
    axes[1].invert_yaxis()
    
    plt.tight_layout()
    plt.savefig('income_by_block_groups.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✅ Chart saved as 'income_by_block_groups.png'")
    
    # Also show overall distribution by income bins
    fig, ax = plt.subplots(figsize=(14, 6))
    income_bins = [0, 50000, 75000, 100000, 125000, 150000, 200000, 300000]
    income_labels = ['<$50k', '$50k-$75k', '$75k-$100k', '$100k-$125k', '$125k-$150k', '$150k-$200k', '>$200k']
    gdf['income_bin'] = pd.cut(gdf['median_household_income'], bins=income_bins, labels=income_labels)
    bin_counts = gdf['income_bin'].value_counts().sort_index()
    
    bin_counts.plot(kind='bar', ax=ax, color='#6BCF7F', edgecolor='black', linewidth=0.5)
    ax.set_title('Number of Block Groups by Income Range', fontsize=14, fontweight='bold')
    ax.set_xlabel('Income Range', fontsize=12)
    ax.set_ylabel('Number of Block Groups', fontsize=12)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, v in enumerate(bin_counts):
        ax.text(i, v + 5, str(int(v)), ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('income_distribution_bins.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✅ Distribution chart saved as 'income_distribution_bins.png'")


## 6. Income Heat Map with Train Stations Overlay


In [3]:
if 'gdf' in locals():
    # Set CRS to Web Mercator for basemap
    if gdf.crs is None:
        gdf.set_crs(epsg=4326, inplace=True)
    
    gdf_mercator = gdf.to_crs(epsg=3857)
    
    # Convert train stations to same CRS
    if train_stations_gdf is not None:
        train_stations_mercator = train_stations_gdf.to_crs(epsg=3857)
    
    # Create figure
    fig, ax = plt.subplots(figsize=(18, 12))
    
    # Plot choropleth (block groups)
    gdf_mercator.plot(column='median_household_income', 
                      cmap='YlOrRd', 
                      legend=True,
                      ax=ax,
                      edgecolor='white',
                      linewidth=0.3,
                      alpha=0.8,
                      legend_kwds={'label': 'Median Household Income ($)',
                                  'orientation': 'vertical',
                                  'shrink': 0.8})
    
    # Overlay train stations
    if train_stations_gdf is not None:
        train_stations_mercator.plot(ax=ax, 
                                     color='blue', 
                                     markersize=30, 
                                     marker='s',
                                     edgecolor='white',
                                     linewidth=0.5,
                                     alpha=0.7,
                                     label='Train Stations')
        # Add legend for train stations
        ax.legend(loc='upper right', fontsize=10)
    
    # Add basemap
    try:
        ctx.add_basemap(ax, crs=gdf_mercator.crs, source=ctx.providers.CartoDB.Positron)
    except:
        print("Note: Could not add basemap")
    
    ax.set_title('Income Heat Map with Train Stations Overlay\n' +
                'Train Station Area (1-mile buffer) - Suffolk, Middlesex, and Norfolk Counties, MA', 
                fontsize=16, fontweight='bold', pad=20)
    ax.axis('off')
    
    plt.tight_layout()
    plt.savefig('income_heatmap_with_stations.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✅ Map saved as 'income_heatmap_with_stations.png'")


In [4]:
## 7. Income Distribution by County
if 'gdf' in locals():
    gdf['county_code'] = gdf['GEOID'].str[:5]
    county_names = {'25025': 'Suffolk County', '25017': 'Middlesex County', '25021': 'Norfolk County'}
    gdf['county_name'] = gdf['county_code'].map(county_names)
    
    # Summary by county
    print("Income Summary by County:")
    print("=" * 60)
    summary = gdf.groupby('county_name')['median_household_income'].agg([
        'count', 'mean', 'median', 'min', 'max'
    ]).round(0)
    summary.columns = ['Block Groups', 'Mean Income', 'Median Income', 'Min Income', 'Max Income']
    display(summary)
    
    # Bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    county_medians = gdf.groupby('county_name')['median_household_income'].median()
    county_medians.plot(kind='bar', ax=ax, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
    ax.set_title('Median Household Income by County\n(Train Station Area - 1 mile buffer)', 
                 fontsize=14, fontweight='bold')
    ax.set_ylabel('Median Household Income ($)', fontsize=12)
    ax.set_xlabel('County', fontsize=12)
    ax.tick_params(axis='x', rotation=0)
    plt.tight_layout()
    plt.show()


## 6. Income Distribution Histogram
middlesex country, stuffolk country and norfolkkcountry blcok groups 

In [5]:
if 'gdf' in locals():
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Overall distribution
    axes[0].hist(gdf['median_household_income'], bins=50, edgecolor='black', alpha=0.7)
    axes[0].axvline(gdf['median_household_income'].median(), color='red', 
                    linestyle='--', linewidth=2, label=f'Median: ${gdf["median_household_income"].median():,.0f}')
    axes[0].set_title('Income Distribution (All Counties)', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Median Household Income ($)', fontsize=11)
    axes[0].set_ylabel('Number of Block Groups', fontsize=11)
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # By county
    for county, name in county_names.items():
        county_data = gdf[gdf['county_code'] == county]['median_household_income']
        axes[1].hist(county_data, bins=30, alpha=0.6, label=name, edgecolor='black')
    axes[1].set_title('Income Distribution by County', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Median Household Income ($)', fontsize=11)
    axes[1].set_ylabel('Number of Block Groups', fontsize=11)
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
